In [35]:
import os, sys
from google.colab import drive
drive.mount("/content/drive/")
os.chdir("/content/drive/My Drive/chess_app_testing")
sys.path.append("/content/drive/My Drive/chess_app_testing")

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [36]:
image_size = (32, 32)
batch_size = 32

datagen_white_fields = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = (0.2,0.8),
        shear_range=0.2,
        rescale=1./255,
        horizontal_flip=False,
        fill_mode='nearest')
datagen_black_fields = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = (0.2,0.8),
        shear_range=0.2,
        rescale=1./255,
        horizontal_flip=False,
        fill_mode='nearest')


In [37]:
train_white_fields = datagen_white_fields.flow_from_directory(
    'data/lichess/white_fields',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    seed = 1,
    shuffle=True  
)
test_white_fields = datagen_white_fields.flow_from_directory(
    'data/lichess/white_fields',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    seed = 2,
    shuffle=True  
)

Found 13 images belonging to 13 classes.
Found 13 images belonging to 13 classes.


In [38]:
train_black_fields = datagen_black_fields.flow_from_directory(
    'data/lichess/black_fields',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    seed = 1,
    shuffle=True  
)
test_black_fields = datagen_black_fields.flow_from_directory(
    'data/lichess/black_fields',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    seed = 2,
    shuffle=True  
)

Found 13 images belonging to 13 classes.
Found 13 images belonging to 13 classes.


In [39]:
white_model = keras.Sequential(
    [
        keras.Input(shape=(32,32,1)), # 32x32 grayscale
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dense(13, activation="softmax"), # 13 classes
    ]
)

white_model.summary()

black_model = keras.models.clone_model(white_model)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 2, 2, 128)       

In [49]:
white_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
epochs = 20

history = white_model.fit(
    train_white_fields, 
    epochs=epochs,
    batch_size = 128,
    validation_data=test_white_fields
    )
white_model.save_weights('white_model_lichess.h5') 

Epoch 1/20
1/1 [==============================] - 0s 162ms/step - loss: 0.0367 - categorical_accuracy: 1.0000 - val_loss: 0.0024 - val_categorical_accuracy: 1.0000
Epoch 2/20
1/1 [==============================] - 0s 130ms/step - loss: 0.0737 - categorical_accuracy: 0.9231 - val_loss: 0.0082 - val_categorical_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - 0s 130ms/step - loss: 0.0889 - categorical_accuracy: 0.9231 - val_loss: 0.0096 - val_categorical_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 0s 133ms/step - loss: 0.0308 - categorical_accuracy: 1.0000 - val_loss: 0.0188 - val_categorical_accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 115ms/step - loss: 0.0238 - categorical_accuracy: 1.0000 - val_loss: 4.9510e-04 - val_categorical_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 124ms/step - loss: 0.0705 - categorical_accuracy: 1.0000 - val_loss: 0.0011 - val_categorical_accuracy: 1.0000
Epoch 7/20
1

In [53]:
black_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
epochs = 80

history = black_model.fit(
    train_black_fields, 
    epochs=epochs,
    batch_size = 128,
    validation_data=test_black_fields
    )
black_model.save_weights('black_model_lichess.h5') 

Epoch 1/80
1/1 [==============================] - 1s 887ms/step - loss: 0.1726 - categorical_accuracy: 1.0000 - val_loss: 0.2321 - val_categorical_accuracy: 0.9231
Epoch 2/80
1/1 [==============================] - 0s 123ms/step - loss: 0.5866 - categorical_accuracy: 0.8462 - val_loss: 0.3227 - val_categorical_accuracy: 0.9231
Epoch 3/80
1/1 [==============================] - 0s 119ms/step - loss: 0.7218 - categorical_accuracy: 0.7692 - val_loss: 0.2347 - val_categorical_accuracy: 0.9231
Epoch 4/80
1/1 [==============================] - 0s 121ms/step - loss: 0.7558 - categorical_accuracy: 0.8462 - val_loss: 0.2903 - val_categorical_accuracy: 0.8462
Epoch 5/80
1/1 [==============================] - 0s 124ms/step - loss: 0.4918 - categorical_accuracy: 0.7692 - val_loss: 0.3040 - val_categorical_accuracy: 0.8462
Epoch 6/80
1/1 [==============================] - 0s 135ms/step - loss: 0.2846 - categorical_accuracy: 0.9231 - val_loss: 0.1715 - val_categorical_accuracy: 0.9231
Epoch 7/80
1/1 [

In [54]:
converter_white_model = tf.lite.TFLiteConverter.from_keras_model(white_model)
converter_white_model.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter_white_model.convert()

f = open('white_model_lichess.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

INFO:tensorflow:Assets written to: /tmp/tmpc5vmjd8b/assets


In [55]:
converter_black_model = tf.lite.TFLiteConverter.from_keras_model(black_model)
converter_black_model.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter_black_model.convert()

f = open('black_model_lichess.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

INFO:tensorflow:Assets written to: /tmp/tmpcwqqcjay/assets


INFO:tensorflow:Assets written to: /tmp/tmpcwqqcjay/assets
